In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/ME_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5508457004,ME,"100 Gannett Dr Suite C, SOUTH PORTLAND, ME",43.636154,-70.355718,2.300500e+14
1,8722078706,ME,"50 Moody St, Saco, ME",43.517756,-70.435568,2.303101e+14
2,4478017005,ME,"103 Maine Avenue, BANGOR, ME",44.801223,-68.807735,2.301900e+14
3,4752408810,ME,"901 Washington Ave Ste 100, Portland, ME",43.689191,-70.272333,2.300500e+14
4,4593917108,ME,"324 Gannett Drive,, SOUTH PORTLAND, ME",43.641674,-70.354535,2.300500e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5508457004,ME,"100 Gannett Dr Suite C, SOUTH PORTLAND, ME",43.636154,-70.355718,2.300500e+14
1,8722078706,ME,"50 Moody St, Saco, ME",43.517756,-70.435568,2.303101e+14
2,4478017005,ME,"103 Maine Avenue, BANGOR, ME",44.801223,-68.807735,2.301900e+14
3,4752408810,ME,"901 Washington Ave Ste 100, Portland, ME",43.689191,-70.272333,2.300500e+14
4,4593917108,ME,"324 Gannett Drive,, SOUTH PORTLAND, ME",43.641674,-70.354535,2.300500e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1469,3886797304,ME,"7 Oxford Homes Lane, Oxford, ME",44.150196,-70.482626,NaN
2110,7910567010,ME,"728 MAIN ST, RICHMOND, ME",37.538949,-77.437307,NaN
2129,7939628407,ME,"728 Main St Ste 1, Richmond, ME",40.726457,-73.821711,NaN
3240,7713547005,ME,"159 Flight Dr., Auburn, ME",42.886902,-85.774071,NaN
3272,6202277109,ME,"65 Main St, Richmond, ME",37.542424,-77.443738,NaN
3497,1916827107,ME,"PO BOX 116, BERNARD, ME",45.253783,-69.445469,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('23')] #ME
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/6 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

6


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_14666/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1469,3886797304,ME,"7 Oxford Homes Lane, Oxford, ME",44.150196,-70.482626,[230179664023012]
2110,7910567010,ME,"728 MAIN ST, RICHMOND, ME",37.538949,-77.437307,[]
2129,7939628407,ME,"728 Main St Ste 1, Richmond, ME",40.726457,-73.821711,[]
3240,7713547005,ME,"159 Flight Dr., Auburn, ME",42.886902,-85.774071,[]
3272,6202277109,ME,"65 Main St, Richmond, ME",37.542424,-77.443738,[]
3497,1916827107,ME,"PO BOX 116, BERNARD, ME",45.253783,-69.445469,[230219604002062]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_14666/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1469,3886797304,ME,"7 Oxford Homes Lane, Oxford, ME",44.150196,-70.482626,230179664023012
2110,7910567010,ME,"728 MAIN ST, RICHMOND, ME",37.538949,-77.437307,NaN
2129,7939628407,ME,"728 Main St Ste 1, Richmond, ME",40.726457,-73.821711,NaN
3240,7713547005,ME,"159 Flight Dr., Auburn, ME",42.886902,-85.774071,NaN
3272,6202277109,ME,"65 Main St, Richmond, ME",37.542424,-77.443738,NaN
3497,1916827107,ME,"PO BOX 116, BERNARD, ME",45.253783,-69.445469,230219604002062


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
2110,7910567010,ME,"728 MAIN ST, RICHMOND, ME",37.538949,-77.437307,NaN
2129,7939628407,ME,"728 Main St Ste 1, Richmond, ME",40.726457,-73.821711,NaN
3240,7713547005,ME,"159 Flight Dr., Auburn, ME",42.886902,-85.774071,NaN
3272,6202277109,ME,"65 Main St, Richmond, ME",37.542424,-77.443738,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
1469,3886797304,230179664023012
2110,7910567010,NaN
2129,7939628407,NaN
3240,7713547005,NaN
3272,6202277109,NaN
3497,1916827107,230219604002062


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/ME_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,442678,5508457004,ME,"100 Gannett Dr Suite C, SOUTH PORTLAND, ME",43.636154,-70.355718,2.300500e+14
1,442679,8722078706,ME,"50 Moody St, Saco, ME",43.517756,-70.435568,2.303101e+14
2,442680,4478017005,ME,"103 Maine Avenue, BANGOR, ME",44.801223,-68.807735,2.301900e+14
3,442681,4752408810,ME,"901 Washington Ave Ste 100, Portland, ME",43.689191,-70.272333,2.300500e+14
4,442682,4593917108,ME,"324 Gannett Drive,, SOUTH PORTLAND, ME",43.641674,-70.354535,2.300500e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,442678,5508457004,ME,"100 Gannett Dr Suite C, SOUTH PORTLAND, ME",43.636154,-70.355718,2.300500e+14,NaN
1,442679,8722078706,ME,"50 Moody St, Saco, ME",43.517756,-70.435568,2.303101e+14,NaN
2,442680,4478017005,ME,"103 Maine Avenue, BANGOR, ME",44.801223,-68.807735,2.301900e+14,NaN
3,442681,4752408810,ME,"901 Washington Ave Ste 100, Portland, ME",43.689191,-70.272333,2.300500e+14,NaN
4,442682,4593917108,ME,"324 Gannett Drive,, SOUTH PORTLAND, ME",43.641674,-70.354535,2.300500e+14,NaN
...,...,...,...,...,...,...,...,...
4180,446858,7989807001,ME,"103 Main Street, KINGFIELD, ME",44.950061,-70.149805,2.300797e+14,NaN
4181,446859,7994057006,ME,"2046 State Road, Eliot, ME",43.164896,-70.810369,2.303104e+14,NaN
4182,446860,9416187007,ME,"29 First Street, SCARBOROUGH, ME",43.600102,-70.321973,2.300502e+14,NaN
4183,446861,9616798809,ME,"1349 Washington Ave, Portland, ME",43.698961,-70.286492,2.300500e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,442678,5508457004,ME,"100 Gannett Dr Suite C, SOUTH PORTLAND, ME",43.636154,-70.355718,2.300500e+14,NaN,230050030022007.0
1,442679,8722078706,ME,"50 Moody St, Saco, ME",43.517756,-70.435568,2.303101e+14,NaN,230310052001033.0
2,442680,4478017005,ME,"103 Maine Avenue, BANGOR, ME",44.801223,-68.807735,2.301900e+14,NaN,230190007001057.0
3,442681,4752408810,ME,"901 Washington Ave Ste 100, Portland, ME",43.689191,-70.272333,2.300500e+14,NaN,230050022003008.0
4,442682,4593917108,ME,"324 Gannett Drive,, SOUTH PORTLAND, ME",43.641674,-70.354535,2.300500e+14,NaN,230050030022006.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5508457004,ME,"100 Gannett Dr Suite C, SOUTH PORTLAND, ME",43.636154,-70.355718,230050030022007.0
1,8722078706,ME,"50 Moody St, Saco, ME",43.517756,-70.435568,230310052001033.0
2,4478017005,ME,"103 Maine Avenue, BANGOR, ME",44.801223,-68.807735,230190007001057.0
3,4752408810,ME,"901 Washington Ave Ste 100, Portland, ME",43.689191,-70.272333,230050022003008.0
4,4593917108,ME,"324 Gannett Drive,, SOUTH PORTLAND, ME",43.641674,-70.354535,230050030022006.0
...,...,...,...,...,...,...
4180,7989807001,ME,"103 Main Street, KINGFIELD, ME",44.950061,-70.149805,230079701011047.0
4181,7994057006,ME,"2046 State Road, Eliot, ME",43.164896,-70.810369,230310370003021.0
4182,9416187007,ME,"29 First Street, SCARBOROUGH, ME",43.600102,-70.321973,230050173053044.0
4183,9616798809,ME,"1349 Washington Ave, Portland, ME",43.698961,-70.286492,230050022002005.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
2110,7910567010,ME,"728 MAIN ST, RICHMOND, ME",37.538949,-77.437307,NaN
2129,7939628407,ME,"728 Main St Ste 1, Richmond, ME",40.726457,-73.821711,NaN
3240,7713547005,ME,"159 Flight Dr., Auburn, ME",42.886902,-85.774071,NaN
3272,6202277109,ME,"65 Main St, Richmond, ME",37.542424,-77.443738,NaN


Convert to File

In [19]:
fips_ME = fips_merge_drop

In [20]:
fips_ME.to_csv("../../../data/state_data/geo/geo_fips/23/ME_fips_scraped.csv") 